In [66]:
from PIL import Image
import os
import shutil
from os.path import isfile
from tqdm.notebook import tqdm

In [29]:
# reformat images to 320x320 max
root_path = 'fabric/images/'
target_dim = 320.0

folders = os.listdir(root_path)
folders = [f for f in folders if not isfile(f)]

all_images = []
all_labels = []

for folder in tqdm(folders[:]):
    #print(folder)
    if 'DS_' not in folder:
        for file in tqdm(os.listdir(root_path+folder)[:], leave=0):
#            print(file)
            filename = file.split('.')
            if (len(filename)<2):
                continue
            if ('jpg' not in filename[1]):
                continue
                
            img = Image.open(root_path+folder+'/'+file)
            img.thumbnail((target_dim, target_dim)) 
            if 'png' in str(type(img)).lower():
                img = img.convert('RGB')
            img.save(root_path+folder+'/'+file, 'JPEG', optimize=True)
            all_images.append(root_path+'/'+folder+'/'+file)

            idx = list(folders).index(folder)
            metadata = str(idx)+' 0.5 0.5 1.0 1.0' # use full image
            filelabel = 'fabric/labels/'+folder+'/'+filename[0]+'.txt'
            f = open(filelabel, 'w')
            f.write(metadata)
            f.close()
            all_labels.append('fabric/labels/'+folder+'/'+filename[0]+'.txt')

  0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/293 [00:00<?, ?it/s]

  0%|          | 0/461 [00:00<?, ?it/s]

  0%|          | 0/417 [00:00<?, ?it/s]

  0%|          | 0/504 [00:00<?, ?it/s]

  0%|          | 0/667 [00:00<?, ?it/s]

  0%|          | 0/874 [00:00<?, ?it/s]

  0%|          | 0/515 [00:00<?, ?it/s]

In [30]:
lines = []
lines.append('path: ../datasets/fabric # root dir\n')
lines.append('train: images/train\n')
lines.append('val: images/val\n')
lines.append('test:  # test images (optional)\n')

lines.append('nc: '+str(len(folders))+' # number of classes\n')

str_classes = "[ '" + "',\n'".join(folders) + "' ]"

lines.append('names: '+str_classes + '\n')

f = open('fabric/fabric.yaml', 'w')
f.writelines(lines)
f.close()

In [32]:
from sklearn.model_selection import train_test_split

train_images, val_images, train_labels, val_labels = train_test_split(all_images, all_labels, test_size=0.2, random_state=314)

In [67]:
def moveFiles(files, type_='images/', dest_='train/'):
    for file in files:
        dest_name = file.replace(type_, type_+dest_)        

        root_folder = '/'.join(file.split('/')[:-2])+'/'+dest_
        if not os.path.exists(root_folder):
            os.mkdir(root_folder)

        folder = '/'.join(file.split('/')[:-1])
        folder = folder.replace(type_, type_+dest_)        
        if not os.path.exists(folder):
            os.mkdir(folder)
            
        try:
            shutil.move(file, dest_name)
        except:
            print('error on ',file)

In [69]:
#moveFiles(train_images, type_='images/', dest_='train/')
moveFiles(train_labels, type_='labels/', dest_='train/')
moveFiles(val_images, type_='images/', dest_='val/')
moveFiles(val_labels, type_='labels/', dest_='val/')